In [1]:
import pandapower as pp
import pandas as pd
import numpy as np
from pandapower.timeseries.output_writer import OutputWriter


# Install pandapower in a new environment by pip install pandapower==2.13.1
net = pp.from_pickle('./aggregated_grid_2018_with_generators_loads_costs_controllers.p')

in_path = "./AC_OPF/Results V3"
out_path = "./AC_OPF_Test"
ow2 = OutputWriter(net, output_path=out_path, output_file_type=".csv", csv_separator=",")
# ow.remove_log_variable("res_gen", "vm_pu")

In [2]:
gen_p_mw = pd.read_csv(in_path + "/res_gen/p_mw.csv").iloc[:, 1:]
gen_vm_pu = pd.read_csv(in_path + "/res_gen/vm_pu.csv").iloc[:, 1:]

load_p_mw = pd.read_csv(in_path + "/res_load/p_mw.csv").iloc[:, 1:]
load_q_mvar = pd.read_csv(in_path + "/res_load/q_mvar.csv").iloc[:, 1:]

sgen_p_mw = pd.read_csv(in_path + "/res_sgen/p_mw.csv").iloc[:, 1:]
sgen_q_mvar = pd.read_csv(in_path + "/res_sgen/q_mvar.csv").iloc[:, 1:]

ext_grid_vm_pu = pd.read_csv(in_path + "/res_bus/vm_pu.csv").iloc[:, 36:]
ext_grid_va_degree = pd.read_csv(in_path + "/res_bus/va_degree.csv").iloc[:, 36:]

In [3]:
final_ext_grid_vm_pu = pd.concat([ext_grid_vm_pu.iloc[:, 3:], ext_grid_vm_pu.iloc[:, :3]], axis=1) # Reorder to bus 38, ..., 35, 36, 37
final_ext_grid_vm_pu.columns = [f"{i}" for i in range(len(net.ext_grid.index))]

final_ext_grid_va_degree = pd.concat([ext_grid_va_degree.iloc[:, 3:], ext_grid_va_degree.iloc[:, :3]], axis=1)
final_ext_grid_va_degree.columns = [f"{i}" for i in range(len(net.ext_grid.index))]

final_ext_grid_vm_pu.head()

start_time = 6912
total_time = 72

In [4]:
def create_controller(df, df_name, df_unit, start_time=0, total_time=8760):
    for prof_name, column in df.items():
        
        if len(column) < 8760:
            before = pd.Series(np.zeros(start_time))
            after = pd.Series(np.zeros(8760-(start_time + total_time)))
            column = pd.concat([before, column, after], ignore_index=True)
            column.name = prof_name

        dataset = column.to_frame()

        dataset = pp.timeseries.data_sources.frame_data.DFData(dataset)

        pp.control.controller.const_control.ConstControl(net, f"{df_name}", f"{df_unit}", element_index=int(prof_name), 
                                                 data_source=dataset, 
                                                 recycle={'trafo': False, 'gen': False, 'bus_pq': True}, 
                                                 profile_name=prof_name)

In [ ]:
# REMOVE CONTROLLERS

# net.controller = net.controller.drop([i for i in range(549, len(net.controller.index))])
# net.controller

In [5]:
create_controller(gen_p_mw, "gen", "p_mw", start_time, total_time)
create_controller(gen_vm_pu, "gen", "vm_pu", start_time, total_time)
create_controller(load_q_mvar, "load", "q_mvar", start_time, total_time)
create_controller(sgen_p_mw, "sgen", "p_mw", start_time, total_time)
create_controller(sgen_q_mvar, "sgen", "q_mvar", start_time, total_time)
create_controller(final_ext_grid_vm_pu, "ext_grid", "vm_pu", start_time, total_time)
create_controller(final_ext_grid_va_degree, "ext_grid", "va_degree", start_time, total_time)

In [ ]:
# LOG SIMULATION VARIABLES 2, TRY OUT LOGGING CURRENT

out_path2 = "./AC_OPF_Test/Checking_current"

ow2 = OutputWriter(net, output_path=out_path2, output_file_type=".csv", csv_separator=",")

ow2.log_variable('res_bus', 'vm_pu') # Add logging for Bus
ow2.log_variable('res_bus', 'va_degree') 
ow2.log_variable('res_bus', 'p_mw') 
ow2.log_variable('res_bus', 'q_mvar')

ow2.log_variable('res_line', 'loading_percent') # Line
ow2.log_variable('res_line', 'i_from_ka') # Line
ow2.log_variable('res_line', 'i_to_ka') # Line
ow2.log_variable('res_line', 'i_ka') # Line
ow2.log_variable('res_line', 'p_from_mw') # Line
ow2.log_variable('res_line', 'q_from_mvar') # Line
ow2.log_variable('res_line', 'p_to_mw') # Line
ow2.log_variable('res_line', 'q_to_mvar') # Line
ow2.log_variable('res_line', 'vm_from_pu') # Line
ow2.log_variable('res_line', 'vm_to_pu') # Line
ow2.log_variable('res_line', 'va_from_degree') # Line
ow2.log_variable('res_line', 'va_to_degree') # Line

ow2.log_variable('res_gen', 'p_mw') # Gen
ow2.log_variable('res_gen', "q_mvar")
ow2.log_variable('res_gen', "va_degree")
ow2.log_variable('res_gen', "vm_pu")

ow2.log_variable('res_sgen', 'p_mw') # Sgen
ow2.log_variable('res_sgen', 'q_mvar')

ow2.log_variable('res_load', 'p_mw') # Load
ow2.log_variable('res_load', 'q_mvar')

ow2.log_variable('res_ext_grid', 'p_mw') # Ext grid
ow2.log_variable('res_ext_grid', 'q_mvar')

ow2.log_variable('res_trafo', 'loading_percent') # Transformer
ow2.log_variable('res_trafo', 'p_hv_mw')
ow2.log_variable('res_trafo', 'p_hv_mvar')
ow2.log_variable('res_trafo', 'p_lv_mw')
ow2.log_variable('res_trafo', 'q_lv_mvar')
ow2.log_variable('res_trafo', 'pl_mw')
ow2.log_variable('res_trafo', 'ql_mvar')
ow2.log_variable('res_trafo', 'i_hv_ka')
ow2.log_variable('res_trafo', 'i_lv_ka')
ow2.log_variable('res_trafo', 'vm_hv_pu')
ow2.log_variable('res_trafo', 'va_hv_degree')
ow2.log_variable('res_trafo', 'vm_lv_pu')
ow2.log_variable('res_trafo', 'va_lv_degree')

In [6]:
# TRY OUT LOGGING CURRENT

pp.timeseries.run_timeseries(net, time_steps=range(6912, 6912+24), run=pp.runpp)

c:\Users\bjorn\miniconda3\lib\site-packages\pandapower\timeseries\output_writer.py:177: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'range(6912, 6936)' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  self.output["Parameters"].loc[:, "time_step"] = self.time_steps
c:\Users\bjorn\miniconda3\lib\site-packages\pandapower\control\run_control.py:50: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  level = controller.level.fillna(0).apply(asarray).values
100%|██████████| 24/24 [00:05<00:00,  4.58it/s]


In [7]:
# TIMESTEP-WISE SIMULATION TO GET SIMULATED VALUES

# Simulate for a single timestep (t) using function run (pp.runpp, pp.runopp,...) and give the outputs at net.res_...
def get_simulated_values(t, run):
    for controller in net.controller.object:
        controller.time_step(net, t)

    run(net, verbose=True)

get_simulated_values(6912, run=pp.runpp)

In [8]:
net.res_ext_grid

,p_mw,q_mvar
0,0.001624,20.508069
1,0.001616,-7.808373
2,0.000800,116.071249
3,0.000044,-2.454733
4,0.000080,-2.453964
5,0.000349,-3.778463
6,-0.001561,-3.087029
7,0.001051,-377.214676
8,0.000000,0.000000
9,-0.003158,0.104310


In [9]:
net.res_line[90:94] # Bus voltages, p_mw, va_degree are the same, q_mvar more or less same
# Ext_grid p_mw increased a lot, q_mvar more or less same

,p_from_mw,q_from_mvar,p_to_mw,q_to_mvar,pl_mw,ql_mvar,i_from_ka,i_to_ka,i_ka,vm_from_pu,va_from_degree,vm_to_pu,va_to_degree,loading_percent
90,0.000525,-188.607338,-0.000183,188.567617,0.000342,-0.039721,0.286559,0.28649,0.286559,1.0,0.0,1.00003,-0.000104,7.16398
91,0.000525,-188.607338,-0.000183,188.567617,0.000342,-0.039721,0.286559,0.28649,0.286559,1.0,0.0,1.00003,-0.000104,7.16398


In [10]:
net.res_bus

,vm_pu,va_degree,p_mw,q_mvar
0,1.038655,1.466232,0.000000,0.000000
1,1.029049,-2.111461,0.000000,0.000000
2,1.031064,-1.972435,77.410808,2.876487
3,1.032532,-1.669895,175.079668,-5.360625
4,1.035399,0.826483,153.960297,50.604302
5,1.030620,-1.900843,23.537389,21.220668
6,1.038637,1.457302,215.888998,17.745116
7,1.035664,-0.207765,0.000000,0.000000
8,1.037715,1.106987,29.137285,-11.009924
9,1.034367,-0.840908,55.199233,11.255025
